In [1]:
import pandas as pd
import numpy as np
import gc
import timeit
from numpy.linalg import inv

In [17]:
start_time = timeit.default_timer()
df = pd.read_csv("/datasets/iowaliquor.csv")
df = df.iloc[::10, :]

elapsed = timeit.default_timer() - start_time
print("Time (minutes) elapsed for this cell:", elapsed/60)

df.head()

/opt/tljh/user/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Time (minutes) elapsed for this cell: 0.8143883617166617


,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S08096000008,10/04/2012,4641,Kum & Go #573 / SE 14th DM,5830 SE 14th ST,DES MOINES,50315,NaN,77.0,Polk,...,89191,Jose Cuervo Especial Reposado Tequila Mini,12,500,11.50,17.25,1,17.25,0.5,0.13
10,S05372600066,05/03/2012,2527,Hy-Vee Food Store #5 / Des Moines,1107 SE ARMY POST RD,DES MOINES,50315,POINT (-93.602372 41.526289),77.0,Polk,...,43026,Admiral Nelson Spiced Rum,12,750,5.77,8.66,24,207.84,18.0,4.76
20,S07363200013,08/28/2012,2662,Hy-Vee Wine & Spirits / Muscatine,"522 MULBERRY, SUITE A",MUSCATINE,52761,POINT (-91.04437 41.426615),70.0,Muscatine,...,37418,Popov Vodka 80 Prf,6,1750,8.49,12.74,12,152.88,21.0,5.55
30,S18612600022,04/24/2014,3437,Scott's Foods,1120 SUNSET DRIVE,NORWALK,50211,POINT (-93.67803700000002 41.479944),91.0,Warren,...,54646,Arrow Blackberry Flav Brandy,12,750,5.38,8.06,12,96.72,9.0,2.38
40,S21013800004,09/03/2014,2596,Hy-Vee / Regal Liquors and Video,2453 NORTH COURT,OTTUMWA,52501,POINT (-92.408085 41.049854),90.0,Wapello,...,42141,UV Sugar Crush Mini,1,3000,29.30,43.95,1,43.95,3.0,0.79


In [18]:
# CLEANING ALCOHOL CATEGORICAL VARIABLES
df['Is_WHISKEY'] = df['Category Name'].str.contains('WHISK')
df['Is_VODKA'] = df['Category Name'].str.contains('VODKA')
df['Is_RUM'] = df['Category Name'].str.contains('RUM')
df['Is_SCHNAPPS'] = df['Category Name'].str.contains('SCHNAPPS')
df['Is_BRANDY'] = df['Category Name'].str.contains('BRAND')
df['Is_GIN'] = df['Category Name'].str.contains('GIN')
df["Is_TEQUILA"] = df['Category Name'].str.contains('TEQUILA')
df['Consolidated'] = pd.Series
#df["Consolidated"] = ""

df.loc[df['Is_WHISKEY'] == True, 'Consolidated'] = 'Whiskey'
df.loc[df['Is_VODKA'] == True, 'Consolidated'] = 'Vodka'
df.loc[df['Is_RUM'] == True, 'Consolidated'] = 'Rum'
df.loc[df['Is_SCHNAPPS'] == True, 'Consolidated'] = 'Schnapps'
df.loc[df['Is_BRANDY'] == True, 'Consolidated'] = 'Brandy'
df.loc[df['Is_GIN'] == True, 'Consolidated'] = 'Gin'
df.loc[df['Is_TEQUILA'] == True, 'Consolidated'] = 'Tequila'
df.loc[df['Consolidated'] == "",'Consolidated'] = 'Other'

In [19]:
df.head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Volume Sold (Liters),Volume Sold (Gallons),Is_WHISKEY,Is_VODKA,Is_RUM,Is_SCHNAPPS,Is_BRANDY,Is_GIN,Is_TEQUILA,Consolidated
0,S08096000008,10/04/2012,4641,Kum & Go #573 / SE 14th DM,5830 SE 14th ST,DES MOINES,50315,NaN,77.0,Polk,...,0.5,0.13,False,False,False,False,False,False,True,Tequila
10,S05372600066,05/03/2012,2527,Hy-Vee Food Store #5 / Des Moines,1107 SE ARMY POST RD,DES MOINES,50315,POINT (-93.602372 41.526289),77.0,Polk,...,18.0,4.76,False,False,True,False,False,False,False,Rum
20,S07363200013,08/28/2012,2662,Hy-Vee Wine & Spirits / Muscatine,"522 MULBERRY, SUITE A",MUSCATINE,52761,POINT (-91.04437 41.426615),70.0,Muscatine,...,21.0,5.55,False,True,False,False,False,False,False,Vodka
30,S18612600022,04/24/2014,3437,Scott's Foods,1120 SUNSET DRIVE,NORWALK,50211,POINT (-93.67803700000002 41.479944),91.0,Warren,...,9.0,2.38,False,False,False,False,True,False,False,Brandy
40,S21013800004,09/03/2014,2596,Hy-Vee / Regal Liquors and Video,2453 NORTH COURT,OTTUMWA,52501,POINT (-92.408085 41.049854),90.0,Wapello,...,3.0,0.79,False,True,False,False,False,False,False,Vodka


In [20]:
df['County'] = df['County'].str.lower()
df.loc[df["County"]=="buena vist","County"] = "buena vista"
df.loc[df["County"]=="cerro gord","County"] = "cerro gordo"
df.loc[df["County"]=="obrien","County"] = "o'brien"
df.loc[df["County"]=="pottawatta","County"] = "pottawattamie"
df["County"] = df["County"].str.capitalize()

In [21]:
df.head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Volume Sold (Liters),Volume Sold (Gallons),Is_WHISKEY,Is_VODKA,Is_RUM,Is_SCHNAPPS,Is_BRANDY,Is_GIN,Is_TEQUILA,Consolidated
0,S08096000008,10/04/2012,4641,Kum & Go #573 / SE 14th DM,5830 SE 14th ST,DES MOINES,50315,NaN,77.0,Polk,...,0.5,0.13,False,False,False,False,False,False,True,Tequila
10,S05372600066,05/03/2012,2527,Hy-Vee Food Store #5 / Des Moines,1107 SE ARMY POST RD,DES MOINES,50315,POINT (-93.602372 41.526289),77.0,Polk,...,18.0,4.76,False,False,True,False,False,False,False,Rum
20,S07363200013,08/28/2012,2662,Hy-Vee Wine & Spirits / Muscatine,"522 MULBERRY, SUITE A",MUSCATINE,52761,POINT (-91.04437 41.426615),70.0,Muscatine,...,21.0,5.55,False,True,False,False,False,False,False,Vodka
30,S18612600022,04/24/2014,3437,Scott's Foods,1120 SUNSET DRIVE,NORWALK,50211,POINT (-93.67803700000002 41.479944),91.0,Warren,...,9.0,2.38,False,False,False,False,True,False,False,Brandy
40,S21013800004,09/03/2014,2596,Hy-Vee / Regal Liquors and Video,2453 NORTH COURT,OTTUMWA,52501,POINT (-92.408085 41.049854),90.0,Wapello,...,3.0,0.79,False,True,False,False,False,False,False,Vodka


In [22]:
# DATA CLEANING/ORGANIZATION
start_time = timeit.default_timer()

# only include the features we need
df = df[["Date", "County", "Pack", "Consolidated", 
         "Vendor Name", "State Bottle Cost", "State Bottle Retail", 
         "Bottles Sold", "Sale (Dollars)", "Volume Sold (Gallons)"]]

df.dropna(inplace=True)

gc.collect()

# type casting
categorical_cols = ["Consolidated", "Vendor Name"]
for col in categorical_cols:
    df[col] = df[col].astype('category')
    
# get year from date
df['Date'] = pd.to_datetime(df['Date'])
df.insert(1, 'Year', df['Date'].dt.year)

gc.collect()

# get dummies for categorical variables
county_var = df["County"]
#df = pd.get_dummies(df.drop(["County"], axis=1))
df["County"] = county_var

gc.collect()

elapsed = timeit.default_timer() - start_time
print("Time (minutes) elapsed for this cell:", elapsed/60) 

Time (minutes) elapsed for this cell: 0.032302070566667375


In [23]:
df.drop("Vendor Name",axis = 1, inplace = True)

In [24]:
df.head()

,Date,Year,County,Pack,Consolidated,State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Gallons)
0,2012-10-04,2012,Polk,12,Tequila,11.50,17.25,1,17.25,0.13
10,2012-05-03,2012,Polk,12,Rum,5.77,8.66,24,207.84,4.76
20,2012-08-28,2012,Muscatine,6,Vodka,8.49,12.74,12,152.88,5.55
30,2014-04-24,2014,Warren,12,Brandy,5.38,8.06,12,96.72,2.38
40,2014-09-03,2014,Wapello,1,Vodka,29.30,43.95,1,43.95,0.79


In [25]:
# AGGREGATION BY COUNTY AND YEAR
start_time = timeit.default_timer()

df.insert(1, 'Month-Year', df['Date'].dt.strftime('%m-%Y'))
df.drop(["Date", "Year"], axis=1, inplace=True)
county_var = df["County"]
month_year = df["Month-Year"]
df = pd.get_dummies(df.drop(["County", "Month-Year"], axis=1))
df["County"] = county_var
df["Month-Year"] = month_year


In [31]:
df.drop(["Consolidated_<class 'pandas.core.series.Series'>"], axis=1, inplace=True)

In [32]:
df.head()

,Pack,State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Gallons),Consolidated_Brandy,Consolidated_Gin,Consolidated_Rum,Consolidated_Schnapps,Consolidated_Tequila,Consolidated_Vodka,Consolidated_Whiskey,County,Month-Year
0,12,11.50,17.25,1,17.25,0.13,0,0,0,0,1,0,0,Polk,10-2012
10,12,5.77,8.66,24,207.84,4.76,0,0,1,0,0,0,0,Polk,05-2012
20,6,8.49,12.74,12,152.88,5.55,0,0,0,0,0,1,0,Muscatine,08-2012
30,12,5.38,8.06,12,96.72,2.38,1,0,0,0,0,0,0,Warren,04-2014
40,1,29.30,43.95,1,43.95,0.79,0,0,0,0,0,1,0,Wapello,09-2014


In [33]:
df3 = df
#df = df3

In [34]:
df = df.groupby(['County', 'Month-Year']).agg('mean')

# add the year column back to the data frame
df.reset_index(level=["County", "Month-Year"], inplace=True)
df.insert(1, "Year", df["Month-Year"].str.replace("..-", ""))
df["Year"] = df["Year"].astype(int)

elapsed = timeit.default_timer() - start_time
print("Time (minutes) elapsed for this cell:", elapsed/60) 

Time (minutes) elapsed for this cell: 1.4585840883500016


In [35]:
df.head()

,County,Year,Month-Year,Pack,State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Gallons),Consolidated_Brandy,Consolidated_Gin,Consolidated_Rum,Consolidated_Schnapps,Consolidated_Tequila,Consolidated_Vodka,Consolidated_Whiskey
0,Adair,2012,01-2012,10.411765,7.212647,10.852941,6.823529,72.584412,1.997059,0.000000,0.029412,0.088235,0.176471,0.000000,0.235294,0.323529
1,Adair,2013,01-2013,11.250000,8.291667,12.437083,7.041667,84.189167,1.775000,0.083333,0.083333,0.083333,0.041667,0.000000,0.250000,0.416667
2,Adair,2014,01-2014,11.277778,8.812500,13.218333,9.138889,104.016111,2.127222,0.083333,0.055556,0.000000,0.027778,0.027778,0.194444,0.500000
3,Adair,2015,01-2015,11.219512,8.578537,12.875854,9.365854,115.315122,2.392927,0.121951,0.097561,0.024390,0.024390,0.024390,0.146341,0.439024
4,Adair,2016,01-2016,12.363636,7.401212,11.111515,8.606061,86.475758,1.920000,0.060606,0.000000,0.030303,0.121212,0.000000,0.242424,0.424242


In [11]:
#pip install --user xlrd

In [36]:
# EXTERNAL DATASET, IOWA POPULATION FOR (county, year) PAIRS
df2 = pd.read_excel("iowa_county.xlsx")

# data cleaning/organization
df2 = df2.iloc[4:103]
df2.drop(["Unnamed: 1", "Unnamed: 2"], axis=1, inplace=True)
df2.rename(columns=
{"table with row headers in column A " +
 "and column headers in rows 3 through " +
 "4 (leading dots indicate sub-parts)": "County",
                    "Unnamed: 3": "2010", 
                    "Unnamed: 4": "2011", 
                    "Unnamed: 5": "2012", 
                    "Unnamed: 6": "2013", 
                    "Unnamed: 7": "2014",
                    "Unnamed: 8": "2015",
                    "Unnamed: 9": "2016",
                    "Unnamed: 10": "2017",
                    "Unnamed: 11": "2018",
                    "Unnamed: 12": "2019"}, inplace=True)
df2.reset_index(drop=True, inplace=True)
     
df2["County"] = df2["County"].str.replace(".", "")
df2["County"] = df2["County"].str.replace(" County, Iowa", "")

# convert df to standard form
df2 = df2.melt(var_name="Year", value_name="Population", id_vars=['County'])

# type casting
df2["County"] = df2["County"].astype('category')
df2["Year"] = df2["Year"].astype('int')
df2["Population"] = df2["Population"].astype('int')

df2.head()

,County,Year,Population
0,Adair,2010,7679
1,Adams,2010,4023
2,Allamakee,2010,14378
3,Appanoose,2010,12856
4,Audubon,2010,6098


In [37]:
# EXTERNAL DATASET, IOWA INCOME PER CAPITA FOR (county, year) PAIRS
df_income = pd.read_csv("income.csv")

df_income = df_income[["Name", "Variable", "Value", "Date"]]
df_income = df_income[df_income["Variable"] == "Per capita personal income"]


df_income.rename(columns={"Name": "County", "Value": "Income Per Capita"}, inplace=True)

df_income["County"] = df_income["County"].str.replace(", IA", "")

df_income['Date'] = pd.to_datetime(df_income['Date'])
df_income["Year"] = df_income["Date"].dt.year

df_income.drop(["Variable", "Date"], axis=1, inplace=True)
df_income.head()

,County,Income Per Capita,Year
2073,Adair,21269,1997
2074,Adair,22305,1998
2075,Adair,23030,1999
2076,Adair,25831,2000
2077,Adair,26276,2001


In [38]:
# MERGE BOTH EXTERAL DATASETS
start_time = timeit.default_timer()

# merge with population values for each (county, year) pair
df = pd.merge(df, df2, on=["County", "Year"])

# per capita normalization
df.insert(6, "Bottles Sold Per Capita", df["Bottles Sold"]/df["Population"])
df.insert(7, "Alcohol Expense Per Capita", df["Sale (Dollars)"]/df["Population"])
df.insert(8, "Volume Sold (Gallons) Per Capita", df["Volume Sold (Gallons)"]/df["Population"])

cols_to_drop = ["Bottles Sold", "Sale (Dollars)", "Volume Sold (Gallons)"]
df.drop(cols_to_drop, axis=1, inplace=True)

# merge with income values for each (county, year) pair
df = pd.merge(df, df_income, on=["County", "Year"])
df.drop(["Year"], axis=1, inplace=True)

elapsed = timeit.default_timer() - start_time
print("Time (minutes) elapsed for this cell:", elapsed/60) 
df.head()

Time (minutes) elapsed for this cell: 0.00031862856666672694


,County,Month-Year,Pack,State Bottle Cost,State Bottle Retail,Bottles Sold Per Capita,Alcohol Expense Per Capita,Volume Sold (Gallons) Per Capita,Consolidated_Brandy,Consolidated_Gin,Consolidated_Rum,Consolidated_Schnapps,Consolidated_Tequila,Consolidated_Vodka,Consolidated_Whiskey,Population,Income Per Capita
0,Adair,01-2012,10.411765,7.212647,10.852941,0.000914,0.009719,0.000267,0.000000,0.029412,0.088235,0.176471,0.00000,0.235294,0.323529,7468,42093
1,Adair,02-2012,10.394737,9.180263,13.802105,0.000927,0.011195,0.000250,0.052632,0.105263,0.052632,0.105263,0.00000,0.184211,0.394737,7468,42093
2,Adair,03-2012,12.307692,7.256923,10.931923,0.000917,0.009475,0.000196,0.076923,0.076923,0.153846,0.153846,0.00000,0.153846,0.346154,7468,42093
3,Adair,04-2012,12.142857,7.265238,10.922857,0.000969,0.010872,0.000235,0.071429,0.166667,0.047619,0.047619,0.00000,0.214286,0.309524,7468,42093
4,Adair,05-2012,10.238095,9.279286,13.990476,0.001011,0.013036,0.000260,0.023810,0.047619,0.000000,0.047619,0.02381,0.285714,0.476190,7468,42093


In [39]:
iowa_month_county = df
iowa_month_county.to_csv("iowa_month_county_main.csv", index=False)

In [40]:
iowa_month_county.head()

,County,Month-Year,Pack,State Bottle Cost,State Bottle Retail,Bottles Sold Per Capita,Alcohol Expense Per Capita,Volume Sold (Gallons) Per Capita,Consolidated_Brandy,Consolidated_Gin,Consolidated_Rum,Consolidated_Schnapps,Consolidated_Tequila,Consolidated_Vodka,Consolidated_Whiskey,Population,Income Per Capita
0,Adair,01-2012,10.411765,7.212647,10.852941,0.000914,0.009719,0.000267,0.000000,0.029412,0.088235,0.176471,0.00000,0.235294,0.323529,7468,42093
1,Adair,02-2012,10.394737,9.180263,13.802105,0.000927,0.011195,0.000250,0.052632,0.105263,0.052632,0.105263,0.00000,0.184211,0.394737,7468,42093
2,Adair,03-2012,12.307692,7.256923,10.931923,0.000917,0.009475,0.000196,0.076923,0.076923,0.153846,0.153846,0.00000,0.153846,0.346154,7468,42093
3,Adair,04-2012,12.142857,7.265238,10.922857,0.000969,0.010872,0.000235,0.071429,0.166667,0.047619,0.047619,0.00000,0.214286,0.309524,7468,42093
4,Adair,05-2012,10.238095,9.279286,13.990476,0.001011,0.013036,0.000260,0.023810,0.047619,0.000000,0.047619,0.02381,0.285714,0.476190,7468,42093
